In [2]:
!pip3 install nltk

In [3]:
!pip3 install transformers

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 57.5 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 6.8 MB 17.4 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip3 install tensorflow-gpu
!pip3 install gpt-2-simple

     |████████████████████████████████| 497.5 MB 23 kB/s 
     |████████████████████████████████| 462 kB 27.7 MB/s 
     |████████████████████████████████| 1.4 MB 35.0 MB/s 
     |████████████████████████████████| 15.7 MB 186 kB/s 
     |████████████████████████████████| 5.8 MB 36.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have

     |████████████████████████████████| 1.3 MB 8.7 MB/s 
  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24576 sha256=e009375fb828fa312ce48bf746431b90036d1cc13427c328023eab61778e60ad
  Stored in directory: /root/.cache/pip/wheels/d6/89/8a/f5de6944286d1ac2658b0caa7eae3c8cda50f770cdc957217f
Successfully built gpt-2-simple
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.


In [1]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import random
import tweepy
import gpt_2_simple as gpt2
import os
import requests
import csv
import tensorflow as tf
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
# TODO: Replace the 'XXXXX's with your consumer key, consumer secret, access token, 
# and access secret for Twitter. These will be used by Tweepy.

# The Consumer Twitter API key and secret can be found on the Twitter Developer Portal

# You must generate the Access token and secret yourself by running the provided
# get_access.py code. (You must also add your consumer key and secret to that file.)

CONSUMER_KEY = "5XCSYuQrM6drq8sUCE3KtqyB5"
CONSUMER_SECRET = "HTgoUyg3uPyjl8IIEnjGk8QLTu9h1l21vi8CUQCK0HfgHTdiZZ"

ACCESS_TOKEN = "1488596326455693314-meMb01EzUGpuoBop1VQXF9Ez1aXHKw"
ACCESS_SECRET = "wHuykQWGzdxn6DgF7d81VQtc08sNSYVVCaXw9nMC9fZNJ"

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
gpt2.download_gpt2(model_name="124M")
gpt2.mount_gdrive()

Fetching checkpoint: 1.05Mit [00:00, 406Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 4.08Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 520Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 50.7Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 569Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 5.91Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.60Mit/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Sets up the Twitter API
def getAPI():
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api

In [4]:
nsteps = 1000

api = getAPI()

In [5]:
# TODO: Implement parts 3 and 4
# (i)
userID = "jayrosen_nyu"
corpus = []
for i in range(5):
    corpus.extend(api.user_timeline(screen_name=userID, count=200, include_rts = False, tweet_mode = 'extended'))
# (ii)
print(len(corpus))
corpus.extend(api.user_timeline(screen_name="marknorm", count=55, include_rts = False, tweet_mode = 'extended'))
print(len(corpus))
corpus.extend(api.user_timeline(screen_name="bertkreischer", count=200, include_rts = False, tweet_mode = 'extended'))
print(len(corpus))
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

945
1000
1163


In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
file_name = "tweets.csv"
if not os.path.isfile(file_name):
  with open(file_name, 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["tweet"])
    for tweet in corpus:
      words = nltk.word_tokenize(tweet.full_text)
      new_words = [word for word in words if word.isalnum()]    
      new_tweet = " ".join([str(item) for item in new_words])
      writer.writerow(new_tweet)

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              steps=nsteps,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              )   # steps is max number of training steps

gpt2.copy_checkpoint_to_gdrive(run_name='run1')

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:00<00:00, 130.03it/s]

dataset has 18608 tokens
Training...


[10 | 1133.08] loss=0.23 avg=0.23


In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-200
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-200


In [ ]:
text = gpt2.generate(sess, run_name='run1', return_as_list=True)[0]

In [ ]:
nltk.download([
        "names",
        "stopwords",
        "state_union",
        "twitter_samples",
        "movie_reviews",
        "averaged_perceptron_tagger",
        "vader_lexicon",
        "punkt",
    ])

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   

True

In [ ]:
freedonia = "FreedoniaNews"

freedonia_tweets_info = (api.user_timeline(screen_name=freedonia, include_rts = False, tweet_mode = 'extended'))
freedonia_tweets = []
for tweet in freedonia_tweets_info:
  freedonia_tweets.append(tweet.full_text)

In [ ]:
freedonia_tweets = [t.replace("://", "//") for t in freedonia_tweets]

In [ ]:
#sia = SentimentIntensityAnalyzer()
sentiments = []
for tweet in freedonia_tweets:
  analysis = TextBlob(tweet)
  polarity = analysis.sentiment.polarity
  if polarity > 0:
    sentiment = "positive"
  elif polarity == 0:
    sentiment = "neutral"
  else:
    sentiment = "negative"
  print(sentiment)
  sentiments.append(sentiment)

positive
negative
negative
positive
negative
negative
positive
positive


In [ ]:
subjects = []
for tweet in freedonia_tweets:
  if "Slyvania" in tweet:
    sub = "Slyvania"
  elif "Ambassador Trentino" in tweet:
    sub = "Ambassador Trentino"
  elif "Freedonia" in tweet:
    sub = "Freedonia"
  elif "Rufus T. Firefly" in tweet:
    sub = "Rufus T. Firefly"
  print(sub)
  subjects.append(sub)

Rufus T. Firefly
Freedonia
Rufus T. Firefly
Rufus T. Firefly
Freedonia
Freedonia
Ambassador Trentino
Rufus T. Firefly


In [ ]:
agrees = {}
agrees["Slyvania"] = True
agrees["Ambassador Trentino"] = True
agrees["Freedonia"] = False
agrees["Rufus T. Firefly"] = False

In [ ]:
response = None
i = random.randrange(8)
response_sentiment = None
tweet_sentiment = sentiments[i]
subject = subjects[i]
if agrees[subject]:
  if tweet_sentiment == "positive":
    desired_sentiment = "positive"
  else:
    desired_sentiment = "negative"
else:
  if tweet_sentiment == "positive":
    desired_sentiment = "negative"
  else:
    desired_sentiment = "positive"
while response_sentiment != desired_sentiment:
  text = gpt2.generate(sess, run_name="run1", prefix=subject, length=40, return_as_list=True)[0]
  analysis = TextBlob(text)
  gen_score = analysis.sentiment.polarity
  if gen_score > 0:
    response_sentiment = "positive"
  elif gen_score < 0:
    response_sentiment = "negative"
print("Found a match for tweet {i}".format(i=i), "The tweet was about {} and had a {} sentiment.\
      The response had a {} sentiment".format(subject, tweet_sentiment, response_sentiment))
print("The response was {}".format(text))
response = text

Found a match for tweet 1 The tweet was about Freedonia and had a negative sentiment.      The response had a positive sentiment
The response was Freedonia@gmail.com<|endoftext|>
<|startoftext|>@johnstodder Okay, but if correcting disinformation is something you want in your 24 hour cable news network


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
freedonia_tweet_id = freedonia_tweets_info[i].id
if len(response) > 279:
  print('here')
  send_it = response[:200]
else:
  send_it = response
print(freedonia_tweet_id)
api.update_status(send_it, freedonia_tweet_id, auto_populate_reply_metadata=True)

1488575899381510144


Status(_api=<tweepy.api.API object at 0x7f339695f5d0>, _json={'created_at': 'Tue Feb 15 04:26:49 +0000 2022', 'id': 1493441644976545793, 'id_str': '1493441644976545793', 'text': '@FreedoniaNews Freedonia@gmail.com&lt;|endoftext|&gt;\n&lt;|startoftext|&gt;@johnstodder Okay, but if correcting disinformation… https://t.co/tnhvg0Z64D', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'FreedoniaNews', 'name': 'Freedonia News Service', 'id': 1078050296113311744, 'id_str': '1078050296113311744', 'indices': [0, 14]}, {'screen_name': 'johnstodder', 'name': 'John Stodder Jr.', 'id': 8395482, 'id_str': '8395482', 'indices': [75, 87]}], 'urls': [{'url': 'https://t.co/tnhvg0Z64D', 'expanded_url': 'https://twitter.com/i/web/status/1493441644976545793', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [128, 151]}]}, 'source': '<a href="https://twitter.com/JackOCo64130255" rel="nofollow">YaleCPSC310Bot</a>', 'in_reply_to_status_id': 1488575899381

In [ ]:
from google.colab import drive
drive.mount('/content/drive')